<a href="https://colab.research.google.com/github/Datamin-D/movie_predict/blob/master/Movie_Titles_Webcrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import xlwt,xlrd
from google.colab import files
import time
import re


In [103]:
#installing selenium, prepare to connect 
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (111 kB/s)
Reading package lists... Done
Reading package lists... Done
Building d

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [104]:
#Boxoffice mojo > extract movie titles by years

# 수집할 영화 연도 범위
released_years = []
released_months = []

years = [released_years.append(i) for i in range(2018,2019)] #연
months = [released_months.append(i) for i in range(12,13)] #월

#for문의 내포. [명령 for i in range(시작,끝,step)]
#print(released_years)

# raw data 수집,parsing, 영화 title저장
movie_list = []
for year in released_years:
  for month in released_months:
    #print(genre)
    raw = requests.get("https://www.boxofficemojo.com/calendar/"+str(year)+"-"+str(month)+"-01/")
      #print(raw)해서 response 200이면 정상.
      #주소니까 str로 해줘야. 

      #print(raw.text)
      #.text하면 --> URL소스코드 알아내기

    html = BeautifulSoup(raw.text,"html.parser")
      #변수 = BeautifulSoup(소스코드, 'html.parser')-->태그 기준으로 파싱해줌
      #print(html)

  # 1. 컨테이너 
    container = html.select("div.a-section.a-spacing-none.mojo-schedule-release-details")
  # 2. Title    
    for cont in container:
      movie_title = cont.select_one("a.a-link-normal h3").text.strip()
      #자식 ">" 자손 " "(띄어쓰기) ex) ul.type01 > li
      #.strip-->양쪽 공백 제거

      movie_list.append(movie_title+" "+"("+str(year)+")")
      #태그이름.class, 태그이름#id
print(len(movie_list))

#movie list 고유값만 추리기. set성질 이용.
my_set = set(movie_list)
movie_titles = list(my_set)
print(len(movie_titles))

#movie_titles 표현형식: 영화이름, (연도) 
movie_titles[8]


76
71


"Jim Henson's Holiday Special with Fraggle Rock and Emmet Otter (2018)"

In [10]:
#영화 이름, 연도 분리.. 근데 이미 IMDb에 구현되어 있을 것. --> abort.
"""
indv_title = []
indv_year =  []
for i in range(len(movie_titles)):
  p = re.compile(r'([\'\",/.a-zA-z0-9_ :!?\+\-\&\#\$\@\%\-\=\^\`\(\)]*) (\([0-9]+\))') #정규표현식으로 영화 이름 분리
  target = movie_list[i]
  s = p.search(target) #match는 처음부터 매칭해보고 없으면 진행 안함. search는 문자열 전체를 검색.(앞이 매칭 안되어도.)
  indv_title.append(s.group(1)) #i번째영화이름
  indv_year.append(s.group(2))  #i번째 영화의 연도(인덱스로 찾을 수 있음.)
  m = p.search(target)
print(indv_year[0])
 #이 코드 바탕으로imdb검색한 후 이름,연도 매칭하도록 코딩ㄱ *
 """

'\nindv_title = []\nindv_year =  []\nfor i in range(len(movie_list)):\n  p = re.compile(r\'([\'",/.a-zA-z0-9_ :!?\\+\\-\\&\\#\\$\\@\\%\\-\\=\\^\\`\\(\\)]*) (\\([0-9]+\\))\') #정규표현식으로 영화 이름 분리\n  target = movie_list[i]\n  s = p.search(target) #match는 처음부터 매칭해보고 없으면 진행 안함. search는 문자열 전체를 검색.(앞이 매칭 안되어도.)\n  indv_title.append(s.group(1)) #i번째영화이름\n  indv_year.append(s.group(2))  #i번째 영화의 연도(인덱스로 찾을 수 있음.)\n  m = p.search(target)\nprint(indv_year[0])\n #이 코드 바탕으로imdb검색한 후 이름,연도 매칭하도록 코딩ㄱ *\n '

In [80]:
m = [1,2]
i = []
n = ["en","d"]
min = [m,i,n]
for i in min: 
  try:
    print(i[0])
  except:
    continue


1
en


In [105]:
# Searching movie webpages in IMDb -->needs to be fixed. better idea? :title 밑에 있는 거로 클릭하도록. 

for i in range(len(movie_titles)):
  url = "https://www.imdb.com/find?q="+str(movie_titles[i])+"&ref_=nv_sr_sm" #movie_title에 영화 이름 1개 들어가야함. 
  wd.get(url)
  #print(wd.page_source)  # results
  try:
    click_movie = wd.find_elements_by_css_selector('td.result_text a') #영화 검색창 후 이름 클릭
    click_movie[0].click() #클릭이 안되는 경우, no result found. pass하게 코딩.
  
    time.sleep(1)
  # 여기부터 영화 개별 웹페이지 진입 성공, 자료수집  
  
#Run_time
    Run_time = wd.find_element_by_css_selector('#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > div > time').text
    print(i,Run_time)
  except NoSuchElementException: #time이 기재되지 않았을 때
    continue
  except IndexError: #영화 정보가 없어서 클릭이 안되면.
    continue 


0 1h 29min
1 1h 27min
2 2h 4min
3 2h 12min
4 1h 54min
5 1h 39min
6 1h 44min
7 1h 45min


KeyboardInterrupt: ignored

In [0]:
"""
    # Writers
    Writers = []
    current_url = wd.current_url # 개별 영화 웹페이지 url

    target = str(current_url)
    p = re.compile(r"(https://www.imdb.com/title/)([a-zA-z]{2}[0-9]{7})(/\?ref_=fn_al_tt_1)")
    m = p.match(target)
    movie_code = m.group(2) #영화코드

    url = "https://www.imdb.com/title/"+movie_code+"/fullcredits/?ref_=tt_ov_st_sm"
    wd.get(url) #See full cast & crew 접근
    container = wd.find_elements_by_css_selector("#fullcredits_content > table:nth-child(4) > tbody > tr>td.name") #Writing Credits 박스 접근. 아래 자식으로 작가 이름 크롤링ㄱ

    for i in range(len(container)):
      Writers.append(container[i].text)
    #print(Writers)
    #if #fullcredits_content h4.dataHeaderWithBorder == Writing Credits 거기 멈춰서 안에 있는 박스 > ..는 복잡.
    #걍 nth-ofchild 기능으로 선택자 처리함.

    # Stars 상위16명까지. -->ㅡmin (2003)시도 완료
    #홀수 번호 배우들 수집 
    Stars_ch = []
    Stars_odd = []
    Stars_even = []
    Stars = []

    try:
      container_odd = wd.find_elements_by_css_selector("#fullcredits_content > table.cast_list > tbody > tr.odd") #홀수번호 배우들 데이터 수집
      for i in range(9): #8번째 홀수까지 반복, 15
        Stars_odd.append(container_odd[i].text)
    except IndexError: 
      pass

    #짝수 번호 배우들 수집
    try:
      container_even = wd.find_elements_by_css_selector("#fullcredits_content > table.cast_list > tbody > tr.even") #짝수번호 배우들 데이터 수집
      for i in range(9): #8번째 짝수까지 반복,16 
        Stars_even.append(container_even[i].text)
    except IndexError: 
      pass
    #홀,짝 합치기
    try:

      for i in range(8):
        Stars_ch.append(Stars_odd[i])
        Stars_ch.append(Stars_even[i])
      #print(Stars_ch) 합치기 완료
    except IndexError: #배우 15명 미만일 시 pass.
      pass


    #배우"...캐릭터"지우기는 정규 표현식으로 처리.
    for i in range(len(Stars_ch)):
      target = Stars_ch[i]
      p = re.compile("[\- a-zA-z0-9]+")
      m = p.match(target)
      Star = m.group().strip()
      Stars.append(Star)
    print("Stars:",Stars) #배우목록
"""
  except:
    continue

In [5]:
  #선택자 실험실.Requests, Selenium + Writers Stars 다 수집 다시.
"""
#requests
raw = requests.get("https://www.imdb.com/title/tt7286456/?ref_=ttrel_rel_tt") #test영화2"min" https://www.imdb.com/title/tt0360796/?ref_=fn_al_tt_2
html = BeautifulSoup(raw.text, "html.parser")# 파싱하는법 BeautifulSoup(request로 가져온 url.text, html.parser)
#퍼올꺼 = html.select_one("선택자").text.strip() 
"""
#selenium
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
url = "https://www.imdb.com/find?q="+str("min (2003)")+"&ref_=nv_sr_sm" #영화 이름으로 검색
wd.get(url)
#wd.find_element_by_css_selector("tag")일케 씀.
click_movie = wd.find_elements_by_css_selector('td.result_text a') #영화 검색창 후 이름 클릭
click_movie[0].click() #개별 페이지 진입
time.sleep(1)


"""
p = re.compile(r"(https://www.imdb.com/title/)([a-zA-z]{2}[0-9]{7})(/\?ref_=fn_al_tt_1)")
m = p.match(h_target)
movie_code = m.group(2) #영화코드

for i in range(len(movie_list)):
  p = re.compile('([a-zA-z]{2})')



#Budget, Gross_USA
  #선택자#titleDetails >.txt-block
container = wd.find_elements_by_css_selector("#titleDetails >.txt-block")

#Budget 

for i in range(len(container)):
  if "Budget:" in container[i].text: #if문에 해당하는 내용이 없다해서 오류가 일어나지는 않음.
    budget = container[i].text
    Budget = budget.split(":")[1]
    break  #break는 조건이 참이면 반복문 종료. cf)continue는 아래의 코드를 실행않고 다음 반복으로 넘어감.
  else:
    Budget = "NA"
#Gross_USA
for i in range(len(container)):
  if "Gross USA:" in container[i].text:
    gusa = container[i].text
    Gross_USA = gusa.split(":")[1]
    print(Gross_USA)
    break


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


['Inom Yon', 'Ahmad Hashim', 'Norkhiriah', 'Yasmin Ahmad', 'Alvin Wong', 'Yuhang Ho']


'\np = re.compile(r"(https://www.imdb.com/title/)([a-zA-z]{2}[0-9]{7})(/\\?ref_=fn_al_tt_1)")\nm = p.match(h_target)\nmovie_code = m.group(2) #영화코드\n\nfor i in range(len(movie_list)):\n  p = re.compile(\'([a-zA-z]{2})\')\n\n\n\n#Budget, Gross_USA\n  #선택자#titleDetails >.txt-block\ncontainer = wd.find_elements_by_css_selector("#titleDetails >.txt-block")\n\n#Budget \n\nfor i in range(len(container)):\n  if "Budget:" in container[i].text: #if문에 해당하는 내용이 없다해서 오류가 일어나지는 않음.\n    budget = container[i].text\n    Budget = budget.split(":")[1]\n    break  #break는 조건이 참이면 반복문 종료. cf)continue는 아래의 코드를 실행않고 다음 반복으로 넘어감.\n  else:\n    Budget = "NA"\n#Gross_USA\nfor i in range(len(container)):\n  if "Gross USA:" in container[i].text:\n    gusa = container[i].text\n    Gross_USA = gusa.split(":")[1]\n    print(Gross_USA)\n    break\n'

In [0]:
#Budget 

for i in range(len(container)):
  if "Budget:" in container[i].text: #if문에 해당하는 내용이 없다해서 오류가 일어나지는 않음.
    budget = container[i].text
    Budget = budget.split(":")[1]
    break  #break는 조건이 참이면 반복문 종료. cf)continue는 아래의 코드를 실행않고 다음 반복으로 넘어감.
  else:
    Budget = "NA"

print(Budget)

#Gross USA

In [0]:
# 선택자 실험실- request로 확인 +추후 try except각각 처리해주기 필수 .
#Year,Rate, Certificate, runtime, Director, Writers, Stars,Certificate, Language,Released_Date. runtime은 다시.(영화페이지 밑에 분으로 나온놈 있음.)-->abort. coz N/A in some pages.

"""
url = "https://www.imdb.com/find?q="+str("Joker")+"&ref_=nv_sr_sm" #movie_title에 영화 이름 1개 들어가야함. 
wd.get(url)
  #print(wd.page_source)  # results
click_movie = wd.find_elements_by_css_selector('td.result_text a') #영화 검색창 후 이름 클릭
click_movie[0].click()
import time
time.sleep(1)
  # 여기부터 영화 개별 웹페이지 진입 성공, 자료수집  
Title = html.select("div.title_wrapper h1").text.strip()
print('Title:',Title)

 = find_element_by_css_selector("").text
 
 #Requests로 조커페이지 불러옴
raw = requests.get("https://www.imdb.com/title/tt7286456/?ref_=ttrel_rel_tt") #test영화2"min" https://www.imdb.com/title/tt0360796/?ref_=fn_al_tt_2
html = BeautifulSoup(raw.text, "html.parser")# 파싱하는법 BeautifulSoup(request로 가져온 url.text, html.parser)
#Title

title = html.select_one("div.title_wrapper > h1").text.strip()
print(title)
#title, year 구분이 되어있는게 아님! re로만 제거가능. 근데 그냥 연도까지 같이 가져와서 타이틀로 지정하기.

#year(released year)
Year = html.select_one("span#titleYear a").text.strip()
print(Year)

#rate(전체 별점) 
Rate =  html.select_one("div.ratingValue span:nth-of-type(1):first-child").text.strip() # nth-of-type(n번째) n은 1부터 시작, first-child하면 부모 모든 요소 중 첫번째 자식만 선택 
print(Rate)


  
#Certificate(제한 연령),Relased date * US거로 다시하기. 밑에서
  #container = html.select("div.subtext")
#Run time
Run_time = html.select_one("div.subtext time").text.strip()
print(Run_time)

#Genre - 수집 중 날짜까지 리스트 마지막 수집되어서 index[-1]지우는 식으로 코딩함.
genre = html.select("div.subtext a")[0:-1]
Genre = []
for gen in range(len(genre)):
  Genre.append(genre[gen].text.strip())
print(Genre)

# Director, Writer, Starring div.credit_summary_item줄별로. 
Container = html.select("div.credit_summary_item")

#Director
Director = []
director = Container[0].select("a") 
for i in range(len(director)):
  Director.append(director[i].text.strip())
print(Director)

  # n more credits 지우기
for i in range(len(Director)):
  if "more credits" or "more credit" in Director[i]:
   Ddelete_i = i # Director에서 지워야하는 인덱스
  else: 
    pass
del Director[Ddelete_i] 

# Writers
Writer = []
writer = Container[1].select("a")
for i in range(len(writer)):
  Writer.append(writer[i].text.strip())

  # n more credits 지우기
for i in range(len(Writer)):
  if "more credits" or "more credit" in Writer[i]:
   Wdelete_i = i # Writer에서 지워야하는 인덱스
  else: 
    pass
del Writer[Wdelete_i] 

  # Top2 in Writers 
if len(Writer) > 2: #3명이상이면
  Writer = Writer[0:2]
print(Writer)

# Stars
Stars = []
stars = Container[2].select("a")
for i in range(len(stars)):
  Stars.append(stars[i].text.strip())

  # See full cast & crew 지우기
for i in range(len(Stars)):
  if "See full cast & crew" in Stars[i]:
   Sdelete_i = i # Stars에서 지워야하는 인덱스
  else: 
    pass
del Stars[Sdelete_i] 
  #Top 2 stars
if len(Stars) > 2: #3명이상이면
  Stars = Stars[0:2] # 2명만 extract
print(Stars)

#자동화하기(여러 페이지용.)

#Certificate > clicking see more button using Selenium
url = "https://www.imdb.com/title/tt7286456/?ref_=ttrel_rel_tt" #개별영화 페이지 movie_title에 영화 이름 1개 들어가야함. 
wd.get(url)
#선택자 div.txt-block>span.see-more.inline>a

Certificate_Button = wd.find_element_by_css_selector("div.txt-block>span.see-more.inline>a") #클릭됨?확인
Certificate_Button.click()
time.sleep(1)
#컨테이너 li.ipl-inline-list__item a
countries = wd.find_elements_by_css_selector("li.ipl-inline-list__item a")
for i in range(len(countries)):
  if "United States" in countries[i].text:
    cert = countries[i].text
    cert = cert.split(":")
    Certificate = cert[1]
    #print(Certificate)
    #여기서 마지막 글자만 가져오는 식으로 하자.


#Language: 로 시작하는 컨테이너찾기, 적당히 인덱싱해서 Language찾기. 
container = html.select("div.txt-block")
for i in range(len(container)):
  if container[i].text.strip().startswith("Language:"):
    Lang = container[i].text.strip()
Language = Lang[9:]
#print(Language)


  # Language인 행에서 text가져오기시도. div.txt-block > h4.inline 가
  container = html.select("div.txt-block > h4.inline")
  for i in range(len(container)):
    if container[i].text.strip() == "Language:":
      print(container[i].text.strip())--> #"Language" 나옴


  #그냥 전처리 없이 language:English만 가져오는법
  container = html.select("div.txt-block > a") # >a하면 안되고 > a하니까 됨..
  for i in range(len(container)):
    if container[i].text.strip() == "English":
      print(container[i].text.strip())
  """
"""
#Relased_date
container = html.select("div.txt-block")
for i in range(len(container)):
  if container[i].text.strip().startswith("Release Date:"):
    RDate_list = container[i]
print(RDate_list)



#click span.see-more.inline 눌러야함.
Container = wd.find_elements_by_css_selector("div.txt-block")
for i in range(len(Container)):
  if container[i].text.strip().startswith("Release Date:"):
    RDate_list = Container[i]
print(RDate_list)
RDate_Container = RDate_list.find_elements_by_css_selector("span.see-more.inline") 
Relased_date = RDate_Container.find_element_by_css_selector("a")
print(Relased_date)
#print(RDate_list)
#Rdate_Button.click()


#Released Date(circumvent)
#이거 클릭하면 들어감. Released date div.subtext>a:last-child 
#주의.위에서 url로 클릭한 상태이기에 url초기화해서 다시해줘야.
url = "https://www.imdb.com/title/tt7286456/?ref_=ttrel_rel_tt" #개별영화 페이지 movie_title에 영화 이름 1개 들어가야함. 
wd.get(url)
Released_year = wd.find_elements_by_css_selector("div.subtext > a") 
Released_year[-1].click()
time.sleep(1)
#국가,개봉일 목록 --> tr.ipl-zebra-list__item.release-date-item 
Date_list = wd.find_elements_by_css_selector("tr.ipl-zebra-list__item.release-date-item")

#USA로 시작하는 칸 찾기, 최초 개봉일 extract

for i in range(len(Date_list)):
  if Date_list[i].text.startswith("USA") and "premiere" in Date_list[i].text: #여러 개봉일 중"premiere"개봉일 추출
     premiere_released = i  #인덱스 찾기 성공, td.release-date-item__date 여기로 연계.
     
Date_list = wd.find_elements_by_css_selector("td.release-date-item__date")
Released_Date = Date_list[premiere_released].text #Released_Date 수집 성공
     
#wd.close
"""

Min (2003)


In [0]:
# 엑셀파일 만들기
file_name = 'workbook.xls'
wb = xlwt.Workbook()
ws_1 = wb.add_sheet('sheet0',cell_overwrite_ok = True)
ws_1 = wb.get_sheet(0)
ws_1.write(1,1,'영화제목')

In [0]:
#엑셀파일 다운받기
for i in range(len(new_movie_titles)):
  ws_1.write(i+1,1,new_movie_titles[i])
      
      
wb.save(file_name)
files.download(file_name)